In [ ]:
import numpy as np
import cv2 as cv
import glob
# Критерий для остановки итераций алгоритма (алгоритм остановится при превышении
# 30 итераций или при очень малом смещении пикселей)
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# Подготовка 3D координат
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Массивы, которые будут хранить:
objpoints = [] # 3D координаты из реального мира
imgpoints = [] # 2D координаты из пространства изображения
images = glob.glob('Camera_Calibration/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Находим крайние точки шахматной доски
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # Если нашли, сохраняем данные в массивы
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Рисуем и отображаем получившиеся данные
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
# Калибровка камеры
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
# Предварительная обработка
img = cv.imread('Camera_Calibration/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# Простой способ исправить искажение
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# Создание кропа
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

In [ ]:
# Способ, позволяющий сохранить репрезентацию оригинального изображения
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# Создание кропа
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult1.png', dst)

In [ ]:
# Вычисляем ошибку перепроецирования
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "Значение ошибки: {}".format(mean_error/len(objpoints)) )